In [6]:
# data handling 
import torch 
from torch.utils.data import Dataset, DataLoader
# neural network api
import torch.autograd as autograd 
from torch import Tensor 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

![Transformer architecture](./media/transformer.png)

### Self attention module

<img src="./media/transformer-self-attention.png" alt="self-attention" width="300"/>

In [16]:
class SelfAttention(nn.Module):
    def __init__(self, model_dim, heads ):
        super(SelfAttention, self).__init__()
        self.model_dim = model_dim 
        self.heads = heads 
        
        self.head_dim = model_dim // heads 
        
        assert self.head_dim * heads  == model_dim, f"The model dimensions: {dim_model}, needs to be integer divisible by heads: {heads} "
        
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        # fullying connected output of self attention module
        self.fc_out = nn.Linear(heads * self.head_dim, model_dim )
        
        
    def forward(self, values, keys, query, mask):
        # number of examples 
        N = query.shape[0]
        # these lengths correspond to the intermediate lengths of each input stream 
        # this doesn't vary for this implementation
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        
        # reshape in head pieces 
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = values.reshape(N, keys_len, self.heads, self.head_dim)
        queries = values.reshape(N, query_len, self.heads, self.head_dim)
        
        # queries shape (N, query_len, heads, head_dim)
        # keys shape (N, key_len, heads, head_dim)
        
        # einstein summation notation for tensor multiplication
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # energy shape ( N, heads, query_len, key_len)
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float(-1e20) )
            
        attention = torch.softmax( energy / (self.model_dim**(1/2)),dim = 3)
        # dim=3  -> normalise across the third dim 
        
        # attention shape (N, heads, query_len, key_len)
        # values shape    (N, value_len, heads, heads_dim)        
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
        N, query_len, self.heads*self.head_dim
        )
        # dummy variable l corresponds to key_len and value_len
        out = self.fc_out(out)
        return out
a = SelfAttention(model_dim=12, heads=6)

SelfAttention(
  (values): Linear(in_features=2, out_features=2, bias=False)
  (keys): Linear(in_features=2, out_features=2, bias=False)
  (queries): Linear(in_features=2, out_features=2, bias=False)
  (fc_out): Linear(in_features=12, out_features=12, bias=True)
)

### Transformer block 
<img src="./media/transformer-block.png" alt="transformer-block" width="300"/>

In [23]:
class TransformerBlock(nn.Module):
    def __init__(self, model_dim, heads, dropout, feedforward_dim):
        super(TransformerBlock, self).__init__()
        # init the self attention model
        self.attention = SelfAttention(model_dim, heads)
        
        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(model_dim, feedforward_dim*model_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim*model_dim, model_dim)
        )
        
    def forward(self, value, key, query, mask):
        attention = self.attention(values, key, query, mask)
        # skip connection and dropout
        x = self.dropout(self.norm1(attention + query))
        # feedforward expansion and contraction
        forward = self.feed_forward(x)
        # skip connection and dropout
        out = self.dropout(self.norm2(forward + x))
        return out 
    
t = TransformerBlock(model_dim = 12, heads=2, dropout=0.3, feedforward_dim=4)
t

TransformerBlock(
  (attention): SelfAttention(
    (values): Linear(in_features=6, out_features=6, bias=False)
    (keys): Linear(in_features=6, out_features=6, bias=False)
    (queries): Linear(in_features=6, out_features=6, bias=False)
    (fc_out): Linear(in_features=12, out_features=12, bias=True)
  )
  (norm1): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (feed_forward): Sequential(
    (0): Linear(in_features=12, out_features=48, bias=True)
    (1): ReLU()
    (2): Linear(in_features=48, out_features=12, bias=True)
  )
)

### Encoder
<img src="./media/encoder.png" alt="encoder" width="300"/>

In [33]:
class Encoder(nn.Module):
    
    def __init__(self,
                src_vocab_size, 
                model_dim, 
                heads,
                device,
                feedforward_dim,
                dropout, 
                max_len):
        
        super(Encoder, self).__init__()
        self.model_dim = model_dim 
        self.device = device 
        self.src_vocab_size = src_vocab_size
        # word embeddings
        self.word_emedding =  nn.Embedding(src_vocab_size, model_dim)
        # poisitonal embeddings 
        self.positional_embedding = nn.Embedding(max_len, model_dim)

        self.layers = nn.ModuleList(
        [
            TransformerBlock(model_dim,
                             heads,
                             dropout=dropout,
                             feedforward_dim=feedforward_dim)
        ]
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_len  = x.shape
        
        positions = (torch.arange(0, seq_len)
                     .expand(N, seq_len)
                     .to(self.device))
        # add the word embeddings and the position embeddings together 
        positional_word_embedding = self.word_embedding(x) + self.position_embedding(positions)
        out = self.dropout(position_aware_word_embedding)
        
e = Encoder(100,12, 2, "cpu", 10, 0.3, 10)


### Decoder Block
<img src="./media/decoder.png" alt="decoder" width="300"/>

In [32]:
class DecoderBlock(nn.Module):
    
    def __init__(self, model_dim, heads, feedforward_dim, dropout, device):
        super(DecoderBlock, self).__init__()
        # defining the self attention mechanism 
        self.attention = SelfAttention(model_dim,heads)
        self.norm = nn.LayerNorm(model_dim)
        self.transformer_block = TransformerBlock(
        model_dim, heads, dropout, feedforward_dim
        )
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, value, key, src_mask, trg_mask):
        

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])